# How to Make a ML Model With API Integration
### Step 1 - Acquire Data
<p>Grab a dataset related to the property you would like to try to predict for off of Kaggle, Seaborn, or wherever, I don't care. Here, we'll be using the World Happiness Report (2021) to try to predict how happy a fictional country is.</p>
<p>First, load the dataset csv into the notebook:</p>

In [25]:
import pandas as pd

#Use pandas read_csv module to convert csv file contents to pandas dataframes
df_happiness = pd.read_csv('world-happiness-report.csv')
df_loc = pd.read_csv('world-happiness-report-2021.csv')

<p>If you are using Kaggle, most datasets will have an option to download the CSVs you need directly onto your PC.</p>
<p>In this case, the data is split between two CSVs, which we'll have to fix.</p>

### Cleaning the Data

<p>In order to feed the dataframe into a model, we'll need to adjust its contents a little. Currently, df_happiness looks like this:</p>

In [26]:
df_happiness.keys

<bound method NDFrame.keys of      Country name  year  Life Ladder  Log GDP per capita  Social support  \
0     Afghanistan  2008        3.724               7.370           0.451   
1     Afghanistan  2009        4.402               7.540           0.552   
2     Afghanistan  2010        4.758               7.647           0.539   
3     Afghanistan  2011        3.832               7.620           0.521   
4     Afghanistan  2012        3.783               7.705           0.521   
...           ...   ...          ...                 ...             ...   
1944     Zimbabwe  2016        3.735               7.984           0.768   
1945     Zimbabwe  2017        3.638               8.016           0.754   
1946     Zimbabwe  2018        3.616               8.049           0.775   
1947     Zimbabwe  2019        2.694               7.950           0.759   
1948     Zimbabwe  2020        3.160               7.829           0.717   

      Healthy life expectancy at birth  Freedom to make l

<p>This is largely fine. However, df_loc contains the final score and the geographical location of the country, which we need.</p>

In [30]:
#Restrict to needed columns
df_final = df_happiness[["Country name", "year", "Log GDP per capita", "Social support","Healthy life expectancy at birth", "Freedom to make life choices"]]
df_final

#Create dictionaries matching names to scores and location of all countries
# add_scores = dict(zip(df_loc['Country name'], df_loc['Ladder score']))
# add_locs = dict(zip(df_loc['Country name'], df_loc['Regional indicator']))

,Country name,year,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices
0,Afghanistan,2008,7.370,0.451,50.80,0.718
1,Afghanistan,2009,7.540,0.552,51.20,0.679
2,Afghanistan,2010,7.647,0.539,51.60,0.600
3,Afghanistan,2011,7.620,0.521,51.92,0.496
4,Afghanistan,2012,7.705,0.521,52.24,0.531
...,...,...,...,...,...,...
1944,Zimbabwe,2016,7.984,0.768,54.40,0.733
1945,Zimbabwe,2017,8.016,0.754,55.00,0.753
1946,Zimbabwe,2018,8.049,0.775,55.60,0.763
1947,Zimbabwe,2019,7.950,0.759,56.20,0.632
